# *LinearSolve* Package

## Contents

- [TBD](#tbd)

LinearSolve.jl is a unified interface for the linear solving packages of Julia. It interfaces with other packages of the Julia ecosystem to make it easy to test alternative solver packages and pass small types to control algorithm swapping. It also interfaces with the ModelingToolkit.jl world of symbolic modeling to allow for automatically generating high-performance code.

## But hWhy?

In [1]:
using LinearAlgebra

In [22]:
A = [1. 1. 3.
     4. 5. 6.
     7. 8. 9.]
b1 = [1., 2., 3.]
b2 = [1., 0., 1.];

In [28]:
@time A\b1; A\b2;
@time lu!(A); A\b1; A\b2;

  0.000029 seconds (6 allocations: 304 bytes)
  0.000022 seconds (3 allocations: 112 bytes)


In [43]:
A = [1. 1. 3.
     4. 5. 6.
     7. 8. 9.]
b1 = [1., 2., 3.]
b2 = [1., 0., 1.]
x1 = similar(b1);

In [44]:
@time ldiv!(x1, lu(A), b1)

  0.000029 seconds (5 allocations: 256 bytes)


3-element Vector{Float64}:
  1.4802973661668746e-16
 -1.2952601953960153e-16
  0.3333333333333333

## Solving Linear Systems in Julia

A linear system $Au=b$ is specified by defining an `AbstractMatrix A`, or by providing a matrix-free operator for performing `A*x` operations via the function `A(u,p,t)` out-of-place and `A(du,u,p,t)` for in-place. For the sake of simplicity, this tutorial will only showcase concrete matrices.

The following defines a matrix and a `LinearProblem` which is subsequently solved by the default linear solver.

In [2]:
using LinearSolve

A = rand(4, 4)
b = rand(4)
prob = LinearProblem(A, b)
sol = solve(prob)
sol.u

ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
┌ Warning: Module SciMLBase with build ID ffffffff-ffff-ffff-0004-224e1ee2cf62 is missing from the cache.
│ This may mean SciMLBase [0bca4576-84f4-4d90-8ffe-ffa030f20462] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:2541
┌ Warning: Module LinearSolve with build ID ffffffff-ffff-ffff-0004-224bba9957ed is missing from the cache.
│ This may mean LinearSolve [7ed4a6bd-45f5-4d41-b270-4a48e9bafcae] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:2541


4-element Vector{Float64}:
  1.8526514154446234
 -0.890146462406419
 -0.762372602437811
  1.6852529236449345

Note that `solve(prob)` is equivalent to `solve(prob,nothing)` where nothing denotes the choice of the default linear solver. This is equivalent to the Julia built-in `A\b`, where the solution is recovered via `sol.u`. The power of this package comes into play when changing the algorithms. For example, *Krylov.jl* has some nice methods like *GMRES* which can be faster in some cases. With *LinearSolve.jl*, there is one interface and changing linear solvers is simply the switch of the algorithm choice:

In [45]:
sol = solve(prob, KrylovJL_GMRES())
sol.u

4-element Vector{Float64}:
  1.5659200410393932
 -5.48626286741767
 -4.752664199154221
 10.6899441907668

### Sparse and Structured Matrices

There is no difference in the interface for using LinearSolve.jl on sparse and structured matrices.

In [ ]:
using LinearSolve, SparseArrays

A = sprand(4, 4, 0.75)
b = rand(4)
prob = LinearProblem(A, b)
sol = solve(prob)
sol.u

sol = solve(prob, KrylovJL_GMRES()) # Choosing algorithms is done the same way
sol.u

Similarly structure matrix types, like banded matrices, can be provided using special matrix types. While any AbstractMatrix type should be compatible via the general Julia interfaces, LinearSolve.jl specifically tests a range of cases (see [here](#https://docs.sciml.ai/LinearSolve/stable/tutorials/linear/)).

## Avoiding Refactorization

In [46]:
using LinearSolve

n = 4
A = rand(n, n)
b1 = rand(n);
b2 = rand(n);
prob = LinearProblem(A, b1)

linsolve = init(prob)
sol1 = solve!(linsolve)

retcode: Default
u: 4-element Vector{Float64}:
 -0.46103645603353366
  0.05660653433983502
  0.9790736146374841
  0.5449291529716783

In [47]:
linsolve.b = b2
sol2 = solve!(linsolve)

sol2.u

4-element Vector{Float64}:
  0.9879549541863708
  0.49121666125626523
 -0.579211416259471
 -0.3970125880491276

## Precoditioners

Many linear solvers can be accelerated by using what is known as a **preconditioner**, an approximation to the matrix inverse action which is cheap to evaluate. These can improve the numerical conditioning of the solver process and in turn improve the performance. LinearSolve.jl provides an interface for the definition of preconditioners which works with the wrapped iterative solver packages.

### Mathematical Definition

A right preconditioner, $P_r$ transforms the linear system $Au = b$ into the form:

```math
AP_r^{-1}(P_r u) = AP_r^{-1}y = b
```

which is solved for $y$, and then $P_r u = y$ is solved for $u$. The left
preconditioner, $P_l$, transforms the linear system into the form:

```math
P_l^{-1}Au = P_l^{-1}b
```

A two-sided preconditioned system is of the form:

```math
P_l^{-1}A P_r^{-1} (P_r u) = P_l^{-1}b
```

### Specifying Preconditioners

One way to specify preconditioners uses the `Pl` and `Pr` keyword arguments to `init` or `solve`: 
- `Pl` for left and `Pr` for right preconditioner, respectively. 
- By default, if no preconditioner is given, the preconditioner is assumed to be the identity II.

In the following, we will use a left sided diagonal (Jacobi) preconditioner.

In [8]:
using LinearSolve, LinearAlgebra, BenchmarkTools

n = 4
A = rand(n, n)
b = rand(n)

Pl = Diagonal(A)

prob = LinearProblem(A, b)
sol = solve(prob, KrylovJL_GMRES(), Pl = Pl)
sol.u

4-element Vector{Float64}:
  1.7048359279483938
 -1.059264959257196
  0.8812599920748673
 -1.062083778580742

In [9]:
@btime solve(prob, KrylovJL_GMRES());
@btime solve(prob, KrylovJL_GMRES(), Pl = Pl);

  1.960 μs (50 allocations: 2.44 KiB)
  2.300 μs (54 allocations: 2.56 KiB)


Alternatively, preconditioners can be specified via the `precs` argument to the constructor of an iterative solver specification. This argument shall deliver a factory method mapping `A` and a parameter `p` to a tuple `(Pl,Pr)` consisting a left and a right preconditioner.

In [ ]:
using LinearSolve, LinearAlgebra

n = 4
A = rand(n, n)
b = rand(n)

prob = LinearProblem(A, b)
sol = solve(prob, KrylovJL_GMRES(precs = (A, p) -> (Diagonal(A), I)))
sol.u

This approach has the advantage that the specification of the preconditioner is possible without the knowledge of a concrete matrix A. It also allows to specify the preconditioner via a callable object and to pass parameters to the constructor of the preconditioner instances. The example below also shows how to reuse the preconditioner once constructed for the subsequent solution of a modified problem.

In [ ]:
using LinearSolve, LinearAlgebra

Base.@kwdef struct WeightedDiagonalPreconBuilder
    w::Float64
end

(builder::WeightedDiagonalPreconBuilder)(A, p) = (builder.w * Diagonal(A), I)

n = 4
A = n * I - rand(n, n)
b = rand(n)

prob = LinearProblem(A, b)
sol = solve(prob, KrylovJL_GMRES(precs = WeightedDiagonalPreconBuilder(w = 0.9)))
sol.u

B = A .+ 0.1
cache = sol.cache
reinit!(cache, A = B, reuse_precs = true)
sol = solve!(cache, KrylovJL_GMRES(precs = WeightedDiagonalPreconBuilder(w = 0.9)))
sol.u

### Preconditioners in Julia

Check out [the docs](#https://docs.sciml.ai/LinearSolve/stable/basics/Preconditioners/) for more info.

## Using Matrix-Free Operators via SciMLOperators.jl

In many cases where a sparse matrix gets really large, even the sparse representation cannot be stored in memory. However, in many such cases, such as with PDE discretizations, you may be able to write down a function that directly computes `A*x`. 

In [3]:
function Afunc!(w,v,u,p,t)
    w[1] = -2v[1] + v[2]
    for i in 2:4
        w[i] = v[i-1] - 2v[i] + v[i+1]
    end
    w[5] = v[4] - 2v[5]
    nothing
end

function Afunc!(v,u,p,t)
    w = zeros(5)
    Afunc!(w,v,u,p,t)
    w
end

using SciMLOperators
mfopA = FunctionOperator(Afunc!, zeros(5), zeros(5))

FunctionOperator(5 × 5)

In [6]:
b = rand(5)
prob = LinearProblem(mfopA, b)
sol = solve(prob)
sol.u

5-element Vector{Float64}:
 0.8657410636921089
 0.07604279104377595
 0.07135544053817332
 0.2979002138186635
 0.38842472665875266